# Import the dependencies:

### Installation Required
Run this once to install required packages:
```bash
pip install pandas numpy openpyxl folium
```

In [ ]:
from pathlib import Path
import sys
import os
import re

import pandas as pd
import numpy as np
from tkinter import Tk, filedialog
from math import radians, sin, cos, sqrt, atan2
import folium
from folium import Circle, Marker
from openai import OpenAI

import os

# Load all Data into DataFrame:

In [14]:
#This file contains all the postcode in the UK along with their latitudes and longitudes as exported from link
postcode_db = pd.read_csv('postcodes.csv', low_memory=False)

#### Create an input variable to request postcode from user

In [15]:

input_postcode = input("Please enter a UK postcode(Valid format: SW1A 1AA): ")
radius_miles = float(input("Please enter the radius in miles: "))

Create an Input Variable to Request for `Contact File` from user

In [16]:
# Hide the main Tkinter window
Tk().withdraw()

# Open file dialog
contacts_file = filedialog.askopenfilename(
    title="Select Contacts File",
    filetypes=[("CSV and Excel files", "*.csv *.xlsx")]
)

if not contacts_file:
    raise ValueError("No file selected. Please select a CSV or Excel file.")

# Load file based on type
if contacts_file.endswith('.csv'):
    Contact = pd.read_csv(contacts_file, low_memory=False)
elif contacts_file.endswith('.xlsx'):
    Contact = pd.read_excel(contacts_file, low_memory=False)
else:
    raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

print(f"Loaded {len(Contact)}")

Loaded 862061


In [17]:
# Ask user for the Postcode column name
postcode_column = input("Enter the name of the Postcode column in the Contacts file (default: PostCode): ") or "PostCode"

Clean and Standardize Contact Postcode to valid UK postcode format

In [22]:
# Define a UK postcode cleaner
def clean_uk_postcode(postcode):
    if pd.isna(postcode):
        return None
    postcode = str(postcode).upper().strip()
    postcode = re.sub(r'[^A-Z0-9]', '', postcode)  # remove spaces & non-alphanumerics
    # Insert a space before the last 3 characters if not present
    if len(postcode) > 3:
        postcode = postcode[:-3] + " " + postcode[-3:]
    return postcode.strip()

# Apply to both dataframes
Contact[postcode_column] = Contact[postcode_column].apply(clean_uk_postcode)

In [ ]:
# I want to add additonal step where if not fill the codes can still run
# Ask the user for customer segment column
customer_segment_column = input("Enter the name of the Customer Segment column in the Contacts file (press Enter to skip): ") or None

customer_spend_column = input("Enter the name of the Customer Spend column in the Contacts file (press Enter to skip): ") or None

##### Define an Haversine Function that calculate the distance between the postcode and similar postcode in the database

In [18]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate distance between points in miles"""
    R = 3963.1 # Earth's radius in miles

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    return R * c


postcode_db= postcode_db[['Postcode', 'Latitude', 'Longitude']]


# Use the input postcode directly
center_postcode = input_postcode
print(f"Processing postcode {center_postcode}...")

# Get center postcode coordinates
center = postcode_db[postcode_db['Postcode'] == center_postcode]

if len(center) == 0:
    print(f"Could not find postcode {center_postcode} in the database. Please try again.")
else:
    center = center.iloc[0]
    center_lat = center['Latitude']
    center_lon = center['Longitude']

    # Find postcodes within the specified radius
    radius = radius_miles
    distances = []

    for _, row in postcode_db.iterrows():
        dist = haversine_distance(
            center_lat, center_lon,
            row['Latitude'], row['Longitude']
        )
        if dist <= radius:
            distances.append({
                'Postcode': row['Postcode'],
                'distance_miles': round(dist, 2),
                'area': row['Postcode'].split()[0]  # Add postcode area
            })

    # Create a DataFrame and sort results
    results = pd.DataFrame(distances)
    results = results.sort_values('distance_miles')

    # Display summary of results
    print(f"\nFound {len(results)} postcodes within {radius} miles of {center_postcode}")

Processing postcode BL1 3BL...

Found 47183 postcodes within 10.0 miles of BL1 3BL


In [ ]:
# Contact DataFrame is already loaded from the file dialog cell above
# No need to reload it

# Prepare the postcodes data
postcodes = results.copy()
postcodes = postcodes.rename(columns={'Postcode': 'PostCode'})

# Merge the data based on PostCode
merged_df = pd.merge(Contact, postcodes, left_on= postcode_column, right_on='PostCode', how='inner')


# Save the result to a CSV file
output_file = f'{center_postcode.replace(" ", "_")}_customers.csv'
#merged_df.to_csv(output_file, index=False)

# Count number of unique customers found
customer_count = len(merged_df)

# Display summary
print(f"\nFound {customer_count} unique customers within {radius_miles} miles of {center_postcode}")
print(f"Saved customer data to {output_file}")


Found 6395 unique customers within 10.0 miles of BL1 3BL
Saved customer data to BL1_3BL_customers.csv


#### Display Interactive Map with Coverage Area

In [ ]:
import folium
from folium import Circle, Marker

# Create the map centered on the input postcode
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=10,
    tiles='OpenStreetMap'
)

# Add a marker for the center postcode
folium.Marker(
    [center_lat, center_lon],
    popup=f"<b>{input_postcode}</b><br>Center Point",
    tooltip=input_postcode,
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

# Add a circle showing the radius
folium.Circle(
    location=[center_lat, center_lon],
    radius=radius_miles * 1609.34,  # Convert miles to meters
    color='blue',
    fill=True,
    fillColor='lightblue',
    fillOpacity=0.2,
    popup=f"{radius_miles} mile radius",
    tooltip=f"{radius_miles} miles"
).add_to(m)

# Display the map
st_folium(m, width=700, height=500)

In [ ]:
if (customer_segment_column and customer_segment_column in merged_df.columns) or \
   (customer_spend_column and customer_spend_column in merged_df.columns):

    # --- Case 1: both segment and spend columns exist ---
    if customer_segment_column in merged_df.columns and customer_spend_column in merged_df.columns:
        segment_counts = merged_df[customer_segment_column].value_counts(normalize=True) * 100
        avg_spend = merged_df.groupby(customer_segment_column)[customer_spend_column].mean()

        fig, ax1 = plt.subplots(figsize=(10, 5))

        # Bar chart (percentages)
        bars = ax1.bar(segment_counts.index, segment_counts.values, alpha=0.7, color='skyblue', label="Customer %")
        ax1.set_ylabel("Percentage of Customers (%)")
        ax1.set_xlabel("Customer Segment")
        ax1.set_ylim(0, segment_counts.values.max() * 1.2)

        # Add data labels to bars
        for bar in bars:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2, height + 1,
                     f"{height:.1f}%", ha='center', va='bottom', fontsize=9)

        # Line chart for average spend
        ax2 = ax1.twinx()
        ax2.plot(avg_spend.index, avg_spend.values, color='red', marker='o', label="Average Spend")
        ax2.set_ylabel("Average Spend")
        ax2.tick_params(axis='y', labelcolor='red')

        # Legends
        ax1.legend(loc="upper left")
        ax2.legend(loc="upper right")

        plt.title("Customer Segment Distribution and Average Spend")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

    # --- Case 2: only segment column exists ---
    elif customer_segment_column in merged_df.columns:
        segment_counts = merged_df[customer_segment_column].value_counts(normalize=True) * 100

        plt.figure(figsize=(8, 4))
        bars = plt.bar(segment_counts.index, segment_counts.values, color='skyblue')
        plt.ylabel("Percentage of Customers (%)")
        plt.xlabel("Customer Segment")

        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, height + 1,
                     f"{height:.1f}%", ha='center', va='bottom', fontsize=9)

        plt.title("Customer Segment Distribution")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

    # --- Case 3: only spend column exists ---
    elif customer_spend_column in merged_df.columns:
        plt.figure(figsize=(8, 4))
        plt.plot(merged_df[customer_spend_column], color='red', marker='o')
        plt.ylabel("Spend")
        plt.title("Customer Spend Trend")
        plt.tight_layout()
        plt.show()

